# 03. DeepStream with Vision model

이 강의에서는 ONNX 파일을 TensorRT 엔진 파일로 변환하여 DeepStream 애플리케이션에 배포하는 과정을 안내합니다. TensorRT는 딥러닝 모델의 추론을 최적화하여 고성능, 저지연 결과를 제공합니다.

## 03-2. DeepStream과 TensorRT 개요
- **DeepStream**: NVIDIA의 스트림 기반 비디오 분석 SDK로, TensorRT 엔진 파일을 활용하여 실시간 추론을 지원합니다.
- **TensorRT 엔진 파일**: ONNX 모델에서 변환된 최적화된 파일로, DeepStream에서 추론에 사용됩니다.

### DeepStream 애플리케이션의 기본 구성:
1. 소스 입력 (RTSP, 파일 등)
2. 추론 수행 (TensorRT용 모델 활용)
    * ONNX 모델을 TensorR용 engine파일로 변환해서 사용
3. 결과 시각화 또는 저장

## 03-3. TensorRT와 ONNX
- **TensorRT**: NVIDIA에서 개발한 고성능 딥러닝 추론 최적화 및 런타임 라이브러리.
- **ONNX (Open Neural Network Exchange)**: 다양한 프레임워크 간 상호 운용성을 지원하는 딥러닝 모델 표현 형식.

### TensorRT로 변화하는 이유는?
- DeepStream에서는 TensorRT 모델만 사용이 가능합니다.
- GPU 활용도를 극대화합니다.
- 모델 추론 지연 시간을 줄입니다.
- 추론을 위한 메모리 사용을 최적화합니다.

---

# 실습

## YOLOv11 모델로 engin파일 만들기

### 하드웨어 및 소프트웨어 요구 사항
- TensorRT를 지원하는 NVIDIA GPU.
- TensorRT SDK 설
- Python 환경 (본 실습은 Python 3.10)
- DeepStream 설치(테스트 용도, 선택 사항)

### 라이브러리 및 의존성
Python 환경에서 다음 명령어로 필요한 라이브러리를 설치합니다:
```bash
$ sudo pip3 install onnx onnxruntime tensorrt pycuda
```

### YOLOv11 onnx 모델 다운로드
YOLO 라이브러리 설치 및 numpy 1.23 버전 설치

```bash
$ sudo pip3 install ultralytics
$ sudo pip3 install numpy==1.23
```

#### 1. Python 스크립트를 이용한 YOLOv11.onnx 다운로드

In [1]:
from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO("yolo11n.pt")

# Export the model to ONNX format
model.export(format="onnx")  # creates 'yolo11n.onnx'

# Load the exported ONNX model
onnx_model = YOLO("yolo11n.onnx")

Ultralytics 8.3.65 🚀 Python-3.10.12 torch-2.5.1 CPU (ARMv8 Processor rev 1 (v8l))
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.4 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.47...
ONNX: export success ✅ 4.3s, saved as 'yolo11n.onnx' (10.2 MB)

Export complete (7.0s)
Results saved to /home/paymentinapp/Desktop/lecture/DeepStream/강의자료/03_Vision_Model
Predict:         yolo predict task=detect model=yolo11n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolo11n.onnx imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


#### 2. CLI를 이용한 YOLOv11.onnx 다운로드
```bash
$ yolo export model=yolo11n.pt format=onnx
```

### TensorRT 빌더 스크립트 준비

TensorRT는 ONNX 모델을 엔진 파일로 변환하는 도구를 제공합니다. 아래는 Python 기반의 접근 방식입니다.

In [4]:
import tensorrt as trt

# TensorRT 로거
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

def build_engine(onnx_file_path, engine_file_path):
    # 빌더, 네트워크, 설정 생성
    with trt.Builder(TRT_LOGGER) as builder, \
         builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) as network, \
         trt.OnnxParser(network, TRT_LOGGER) as parser:
        
        # 빌더 구성
        config = builder.create_builder_config()
        config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)  # 1GB 워크스페이스 메모리 설정

        # ONNX 파일 파싱
        with open(onnx_file_path, 'rb') as model:
            if not parser.parse(model.read()):
                print("ONNX 파일 파싱에 실패했습니다.")
                for error in range(parser.num_errors):
                    print(parser.get_error(error))
                return None

        # 네트워크를 직렬화하여 엔진 생성
        serialized_engine = builder.build_serialized_network(network, config)
        if serialized_engine is None:
            print("엔진 생성에 실패했습니다.")
            return None

        # 엔진 파일 저장
        with open(engine_file_path, "wb") as f:
            f.write(serialized_engine)
        print(f"엔진 파일이 {engine_file_path}에 저장되었습니다.")

# 예제 사용법
build_engine("yolo11n.onnx", "yolo11n.engine")


엔진 파일이 yolo11n.engine에 저장되었습니다.


### 엔진 파일 테스트

`.engine` 파일이 생성되면, 올바르게 동작하는지 테스트합니다.

In [5]:
import pycuda.driver as cuda
import pycuda.autoinit
import tensorrt as trt

# TensorRT 엔진 로드
def load_engine(engine_file_path):
    with open(engine_file_path, "rb") as f:
        runtime = trt.Runtime(TRT_LOGGER)
        return runtime.deserialize_cuda_engine(f.read())

engine = load_engine("yolo11n.engine")
print("엔진이 성공적으로 로드되었습니다.")

엔진이 성공적으로 로드되었습니다.


### DeepStream에서 활용
1. **엔진 파일 배치**: `.engine` 파일을 원하는 디렉터리로 이동합니다.
2. **DeepStream 구성 업데이트**: `config_infer_primary.txt` 파일을 수정하여 새 엔진 파일을 사용하도록 설정합니다.

### **구성 파일 예제**
아래는 `config_infer_primary_yolo11n.txt`의 예제입니다:

```txt
[property]
gpu-id=0
model-engine-file=yolo11n.engine
labelfile-path=labels.txt
batch-size=1
network-mode=0
num-detected-classes=80
interval=0
gie-unique-id=1
```

아래는 config_infer_primary_yolo11n.txt를 만드는 코드입니다.

In [8]:
# config_infer_primary_yolo11n.txt 파일 생성
config_content = """[property]
gpu-id=0
model-engine-file=yolo11n.engine
labelfile-path=labels.txt
batch-size=1
network-mode=0
num-detected-classes=80
interval=0
gie-unique-id=1
"""

# 파일 쓰기
with open("config_infer_primary_yolo11n.txt", "w") as f:
    f.write(config_content)

print("config_infer_primary_yolo11n.txt 파일이 생성되었습니다.")


config_infer_primary_yolo11n.txt 파일이 생성되었습니다.


### DeepStream 애플리케이션 코드
이제 Python을 사용하여 DeepStream 애플리케이션을 실행하는 코드를 작성합니다. 이 코드는 TensorRT 엔진 파일을 로드하고, 비디오 스트림에서 추론을 수행합니다.

In [18]:
# 필요한 라이브러리 임포트
import gi
import sys

# GStreamer와 DeepStream 플러그인을 임포트합니다.
gi.require_version('Gst', '1.0')
from gi.repository import Gst

# GStreamer 초기화
Gst.init(None)

[]

In [23]:
# DeepStream 파이프라인 생성
pipeline = Gst.parse_launch(
    "nvv4l2camerasrc device=/dev/video0 ! video/x-raw(memory:NVMM), width=1280, height=720, framerate=30/1 ! "
    "nvvidconv ! video/x-raw(memory:NVMM), format=NV12 ! "
    "mux.sink_0 "
    "nvstreammux name=mux batch-size=1 width=1280 height=720 ! "
    "nvinfer config-file-path=config_infer_primary_yolo11n.txt ! nvtracker ! "
    "nvmultistreamtiler ! nvvideoconvert ! nvdsosd ! nveglglessink"
)

# nvstreammux 설정
mux = pipeline.get_by_name("mux")
if mux:
    mux.set_property("width", 1280)
    mux.set_property("height", 720)
    mux.set_property("batch-size", 1)


Error: gst_parse_error: could not link nvvconv8 to mux, mux can't handle caps video/x-raw(memory:NVMM), format=(string)NV12 (3)

In [11]:
# 파이프라인 실행
def run_pipeline():
    try:
        pipeline.set_state(Gst.State.PLAYING)
        print("DeepStream 파이프라인 실행 중...")

        # GStreamer 버스에서 메시지 수신
        bus = pipeline.get_bus()
        while True:
            msg = bus.timed_pop_filtered(Gst.CLOCK_TIME_NONE, Gst.MessageType.ERROR | Gst.MessageType.EOS)
            if msg:
                t = msg.type
                if t == Gst.MessageType.ERROR:
                    err, debug = msg.parse_error()
                    print(f"에러: {err}, 디버그 정보: {debug}")
                    break
                elif t == Gst.MessageType.EOS:
                    print("End-Of-Stream 도달")
                    break
    except Exception as e:
        print(f"파이프라인 실행 중 오류 발생: {e}")
    finally:
        pipeline.set_state(Gst.State.NULL)

# 실행
run_pipeline()


Using winsys: x11 
DeepStream 파이프라인 실행 중...
에러: gst-library-error-quark: Output width not set (5), 디버그 정보: /dvs/git/dirty/git-master_linux/deepstream/sdk/src/gst-plugins/gst-nvmultistream/gstnvstreammux.cpp(3161): gst_nvstreammux_change_state (): /GstPipeline:pipeline1/GstNvStreamMux:mux
